# Sample static LCIA results


In [1]:
import bw2analyzer as bwa
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import ipywidgets as widgets
import pandas as pd
from bw2data.query import Filter, Query
from IPython.display import display

In [2]:
from project_details import EI_DB_NAME, PROJECT_NAME

In [3]:
bd.projects.set_current(PROJECT_NAME)
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-3.9.1-cutoff

In [4]:
# Is the background database name the same as the one we wrote in `propject_details.py`?
assert EI_DB_NAME in bd.databases

In [5]:
FG_DB_NAME = "asphalt"

In [6]:
db_asphalt = bd.Database(FG_DB_NAME)

In [7]:
pavement_complete_a = db_asphalt.get("DZOAB, A")

In [8]:
bwa.print_recursive_supply_chain(pavement_complete_a, max_level=1)

1: 'DZOAB, A' (kilogram, NL, None)
  1: 'A1, pavement, materials, A' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A' (kilogram, NL, None)
  1: 'A3, pavement, production, A' (kilogram, NL, None)
  1: 'A4, pavement, transport to site' (kilogram, NL, None)
  1: 'A5, pavement, construction' (kilogram, NL, None)
  1: 'C1, pavement, demolition' (kilogram, NL, None)
  1: 'C2, pavement, transport to processing' (kilogram, NL, None)
  1: 'C3, pavement, processing' (kilogram, NL, None)


In [9]:
bwa.print_recursive_supply_chain(pavement_complete_a, max_level=2)

1: 'DZOAB, A' (kilogram, NL, None)
  1: 'A1, pavement, materials, A' (kilogram, NL, None)
    0.3: 'asphalt granulate, free of burden' (kilogram, NL, None)
    0.0412: 'bitumen adhesive compound production, hot' (kilogram, RER, None)
    0.586: 'crushed stone, from quarry in Europe, excluding transport to the Netherlands' (kilogram, NL, None)
    0.0094: 'crushed stone, from quarry in Europe, excluding transport to the Netherlands' (kilogram, NL, None)
    0.0342: 'gravel production, crushed' (kilogram, RoW, None)
    0.0021: 'cellulose fibre production' (kilogram, RoW, None)
    0.027: 'medium filler' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A' (kilogram, NL, None)
    0.0103: 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
    0.547: 'market for transport, freight, sea, ferry' (ton kilometer, GLO, None)
    0.0311: 'market for transport, freight, inland waterways, barge' (ton kilometer, RER, None)
    0.0147: 'market for transport, f

## Find the activities in the db providing "pavement, complete" as ref prod

In [10]:
asphalt_db_data = db_asphalt.load()

In [11]:
query = Query()
filter_refprod = Filter("reference product", "is", "pavement, complete")
query.add(filter_refprod)

In [12]:
res = query(asphalt_db_data)

In [13]:
for res_key, res_act in res.items():
    print(bd.get_activity(res_key))

'DZOAB, B, PVI' (kilogram, NL, None)
'DZOAB, B' (kilogram, NL, None)
'DZOAB, A' (kilogram, NL, None)
'DZOAB, A, PVI' (kilogram, NL, None)


## LCIA

### Functional units

In [14]:
# Use the found activities providing "pavement, complete" as functional units for a multi lca
functional_units = [{bd.get_activity(res_a): 1} for res_a, _ in res.items()]
functional_units

[{'DZOAB, B, PVI' (kilogram, NL, None): 1},
 {'DZOAB, B' (kilogram, NL, None): 1},
 {'DZOAB, A' (kilogram, NL, None): 1},
 {'DZOAB, A, PVI' (kilogram, NL, None): 1}]

In [15]:
# We sort the obtained functional units by code + ref prod
functional_units = sorted(
    functional_units,
    key=lambda fu: [
        list(fu.keys())[0]["code"],
        list(fu.keys())[0]["reference product"],
    ],
)
functional_units

[{'DZOAB, A' (kilogram, NL, None): 1},
 {'DZOAB, A, PVI' (kilogram, NL, None): 1},
 {'DZOAB, B' (kilogram, NL, None): 1},
 {'DZOAB, B, PVI' (kilogram, NL, None): 1}]

### LCIA Methods

In [16]:
ef_methods = [m for m in bd.methods if m[0] == "EF v3.1"]
ef_methods

[('EF v3.1', 'acidification', 'accumulated exceedance (AE)'),
 ('EF v3.1', 'climate change', 'global warming potential (GWP100)'),
 ('EF v3.1', 'climate change: biogenic', 'global warming potential (GWP100)'),
 ('EF v3.1', 'climate change: fossil', 'global warming potential (GWP100)'),
 ('EF v3.1',
  'climate change: land use and land use change',
  'global warming potential (GWP100)'),
 ('EF v3.1',
  'ecotoxicity: freshwater',
  'comparative toxic unit for ecosystems (CTUe)'),
 ('EF v3.1',
  'ecotoxicity: freshwater, inorganics',
  'comparative toxic unit for ecosystems (CTUe)'),
 ('EF v3.1',
  'ecotoxicity: freshwater, organics',
  'comparative toxic unit for ecosystems (CTUe)'),
 ('EF v3.1',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('EF v3.1',
  'eutrophication: freshwater',
  'fraction of nutrients reaching freshwater end compartment (P)'),
 ('EF v3.1',
  'eutrophication: marine',
  'fraction of nutrients reaching marine end compar

In [17]:
bd.calculation_setups["asphalt_pavement_complete"] = {
    "inv": functional_units,
    "ia": ef_methods,
}

In [18]:
# `mlca.results`, is a NumPy array of LCA scores,
# with rows of functional units and
# columns of LCIA methods. Ordering is the same as in the `calculation_setup`.
mlca = bc.MultiLCA("asphalt_pavement_complete")

In [19]:
def format_results(mlca):
    formatted_results = []
    for i, scores in enumerate(mlca.results):  # the results for a fu
        for j, method_key in enumerate(mlca.methods):  # the result for each method
            demand = list(mlca.func_units[i].values())[0]
            activity = list(mlca.func_units[i].keys())[0]
            method = bd.Method(method_key)
            a_result = {
                "activity": repr(activity),
                "demand": demand,
                "method": method_key[0],
                "category": method_key[1],
                "subcategory": method_key[2],
                "score": scores[j],
                "unit": method.metadata["unit"],
            }
            formatted_results.append(a_result)
    return formatted_results

In [20]:
df_results = pd.DataFrame(format_results(mlca))
df_results

,activity,demand,method,category,subcategory,score,unit
0,"'DZOAB, A' (kilogram, NL, None)",1,EF v3.1,acidification,accumulated exceedance (AE),2.347444e-03,mol H+-Eq
1,"'DZOAB, A' (kilogram, NL, None)",1,EF v3.1,climate change,global warming potential (GWP100),1.644502e-01,kg CO2-Eq
2,"'DZOAB, A' (kilogram, NL, None)",1,EF v3.1,climate change: biogenic,global warming potential (GWP100),2.055493e-04,kg CO2-Eq
3,"'DZOAB, A' (kilogram, NL, None)",1,EF v3.1,climate change: fossil,global warming potential (GWP100),1.641591e-01,kg CO2-Eq
4,"'DZOAB, A' (kilogram, NL, None)",1,EF v3.1,climate change: land use and land use change,global warming potential (GWP100),8.557848e-05,kg CO2-Eq
...,...,...,...,...,...,...,...
95,"'DZOAB, B, PVI' (kilogram, NL, None)",1,EF v3.1,material resources: metals/minerals,abiotic depletion potential (ADP): elements (u...,4.327943e-06,kg Sb-Eq
96,"'DZOAB, B, PVI' (kilogram, NL, None)",1,EF v3.1,ozone depletion,ozone depletion potential (ODP),2.074895e-08,kg CFC-11-Eq
97,"'DZOAB, B, PVI' (kilogram, NL, None)",1,EF v3.1,particulate matter formation,impact on human health,4.289613e-08,disease incidence
98,"'DZOAB, B, PVI' (kilogram, NL, None)",1,EF v3.1,photochemical oxidant formation: human health,tropospheric ozone concentration increase,5.372999e-03,kg NMVOC-Eq


In [21]:
df_results.to_excel("asphalt_lcia_results.xlsx")

In [22]:
def filter_fu(fu_name):
    return df_results[df_results.activity == fu_name]

In [23]:
avail_fu_names = [repr(list(fu_a.keys())[0]) for fu_a in functional_units]
select_w = widgets.Dropdown(options=avail_fu_names)
widgets.interact(filter_fu, fu_name=avail_fu_names)

interactive(children=(Dropdown(description='fu_name', options=("'DZOAB, A' (kilogram, NL, None)", "'DZOAB, A, …

<function __main__.filter_fu(fu_name)>